In [54]:
import numpy as np
import pandas as pd

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
df_m = pd.read_csv('/content/drive/MyDrive/DACOS/df_meal.csv')
df_c = pd.read_csv('/content/drive/MyDrive/DACOS/df_cafe.csv')
df_d = pd.read_csv('/content/drive/MyDrive/DACOS/df_drink.csv')

type = int(input("1. 식당, 2. 카페, 3. 술집"))
if type==1:
  df = df_m
elif type==2:
  df = df_c
else:
  df = df_d

,index,가게명,음식종류,음식이 맛있어요,가성비가 좋아요,혼밥하기 좋아요,친절해요,양이 많아요,재료가 신선해요,특별한 메뉴가 있어요,...,컨셉이 독특해요,포장이 깔끔해요,술이 다양해요,직접 잘 구워줘요,커피가 맛있어요,디저트가 맛있어요,기본 안주가 좋아요,집중하기 좋아요,빵이 맛있어요,태그 총합
0,0,육쌈냉면 숙대점,냉면,419.0,165.0,125.0,64.0,64.0,44.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0
1,1,장재근초밥,"초밥,롤",96.0,24.0,16.0,40.0,13.0,56.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,674.0
2,2,면식당 숙명여대점,일식당,373.0,126.0,188.0,128.0,151.0,153.0,68.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3450.0
3,3,포라임 숙대본점,베트남음식,869.0,307.0,419.0,328.0,276.0,327.0,36.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6552.0
4,4,베스트프렌드,종합분식,91.0,21.0,1.0,35.0,12.0,10.0,14.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,514.0


In [ ]:
# 상위 태그 컬럼 제외
if type==1:
  additional_excluded_columns = [
      "음식이 맛있어요",
      '빵이 맛있어요'
      '특별한 메뉴가 있어요',
      '메뉴 구성이 알차요',
      '재료가 신선해요',
      '잡내가 적어요',
      '고기 질이 좋아요'
      "친절해요",
      "가성비가 좋아요",
      "양이 많아요",
      '반찬이 잘 나와요',
      '음식이 빨리 나와요'
      "매장이 청결해요",
      '화장실이 깨끗해요'
  ]
else if type==2:
  additional_excluded_columns = [
      '커피가 맛있어요',
      '디저트가 맛있어요',
      '매장이 청결해요',
      '화장실이 깨끗해요',
      '친절해요',
      '포장이 깔끔해요',
      '아늑해요',
      '인테리어가 멋져요'
    ]
else:
  additional_excluded_columns = [
      '음식이 맛있어요',
      '친절해요',
      '술이 다양해요',
      '음악이 좋아요',
      '대화하기 좋아요'
  ]

# 첫 두 개 컬럼과 마지막 컬럼 제외
excluded_columns = df.columns[:3].tolist() + df.columns[-1:].tolist()

# 최종 제외 리스트
final_excluded_columns = excluded_columns + additional_excluded_columns

# 나머지 컬럼 리스트
remaining_columns = [col for col in df.columns if col not in final_excluded_columns]
print("Remaining Columns:", remaining_columns)

In [59]:
# Input 텍스트와 Remaining Columns 정의
input_txt = [
    '애완동물 동반 가능', '해장하기 좋은 메뉴', '룸이 있는 곳', '단체로 가기 좋은 곳',
    '기념일에 데이트하기 좋은 곳', '조용하고 아늑한 분위기', '든든한 백반',
    '메뉴가 다양함', '맵지 않은 음식', '건강한 한 끼'
]

In [60]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# KoBERT 모델 및 토크나이저 로드
model_name = "monologg/kobert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [61]:
# 문장 임베딩 생성 함수
def get_embeddings(text_list):
    inputs = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # 평균 풀링 사용
    return embeddings

# Threshold 설정
threshold = 0.72

# Input 텍스트와 Remaining Columns 임베딩
input_embeddings = get_embeddings(input_txt)
remaining_embeddings = get_embeddings(remaining_columns)

# 유사도 계산
similarity_matrix = cosine_similarity(input_embeddings.numpy(), remaining_embeddings.numpy())

# 유사도 결과를 데이터프레임으로 변환
similarity_df2 = pd.DataFrame(
    similarity_matrix,
    index=input_txt,
    columns=remaining_columns
)

# high_similarity_pairs 구성 및 결과 저장
high_similarity_pairs = []
result_data = []

for input_text in similarity_df2.index:
    similar_columns = similarity_df2.loc[input_text][similarity_df2.loc[input_text] >= threshold]

    # 상위 5개 선택 (유사도 기준 내림차순)
    selected_tags = similar_columns.sort_values(ascending=False).head(5)

    # 선택된 태그가 있다면 리스트로 저장, 없다면 빈 리스트
    if len(selected_tags) > 0:
        tag_list = selected_tags.index.tolist()
    else:
        tag_list = []

    # high_similarity_pairs에 추가
    for column, similarity in selected_tags.items():
        high_similarity_pairs.append((input_text, column, similarity))

    # 결과 데이터 구성
    result_data.append({"입력 텍스트": input_text, "최종 선택 태그의 리스트": tag_list})

# 결과 데이터프레임 생성
result_df = pd.DataFrame(result_data)

# 출력
print(result_df)

            입력 텍스트                                      최종 선택 태그의 리스트
0       애완동물 동반 가능                                     [반려동물과 가기 좋아요]
1       해장하기 좋은 메뉴  [고기 질이 좋아요, 혼술하기 좋아요, 단체모임 하기 좋아요, 건강한 맛이에요, 특...
2          룸이 있는 곳                  [룸이 잘 되어있어요, 뷰가 좋아요, 특별한 메뉴가 있어요]
3      단체로 가기 좋은 곳  [특별한 날 가기 좋아요, 아이와 가기 좋아요, 현지 맛에 가까워요, 반려동물과 가...
4  기념일에 데이트하기 좋은 곳  [반려동물과 가기 좋아요, 특별한 날 가기 좋아요, 야외공간이 멋져요, 현지 맛에 ...
5     조용하고 아늑한 분위기                                                 []
6           든든한 백반                                                 []
7          메뉴가 다양함                  [룸이 잘 되어있어요, 뷰가 좋아요, 특별한 메뉴가 있어요]
8         맵지 않은 음식  [건강한 맛이에요, 오래 머무르기 좋아요, 음식이 빨리 나와요, 고기 질이 좋아요,...
9          건강한 한 끼                                                 []


In [62]:
# 디버깅: result_df의 컬럼 확인
print("result_df 컬럼들:", result_df.columns)

# 각 input_txt에 대해 조건을 만족하는 가게별 유사도 점수 계산
final_results = []

for input_text in result_df["입력 텍스트"]:
    # 디버깅: 입력 텍스트 확인
    print(f"현재 처리 중인 입력 텍스트: {input_text}")

    # 해당 input_txt의 tag_list 가져오기
    if "최종 선택 태그의 리스트" not in result_df.columns:
        raise KeyError("'최종 선택 태그의 리스트' 컬럼이 존재하지 않습니다. result_df 컬럼들을 확인하세요.")

    tag_list = result_df[result_df["입력 텍스트"] == input_text]["최종 선택 태그의 리스트"].values[0]

    # tag_list가 비어 있는 경우 처리
    if len(tag_list) == 0:
        final_results.append({
            "입력 텍스트": input_text,
            "추천 가게": [],
        })
        continue

    # 태그 수의 비율 계산 (1 / len(tag_list) * 태그 수)
    df["태그 비율"] = df[tag_list].sum(axis=1) / len(tag_list)

    # 조건에 맞는 가게 필터링 (태그 수 >= len(tag_list))
    filtered_df = df[df[tag_list].gt(0).sum(axis=1) >= len(tag_list)]

    # 태그 비율 기준 내림차순 정렬 후 상위 10개 가게 추출
    top_stores = filtered_df.sort_values("태그 비율", ascending=False).head(10)

    # 결과 저장
    recommended_stores = [
        {"가게명": row["가게명"], "유사도 점수": row["태그 비율"]}
        for _, row in top_stores.iterrows()
    ]

    final_results.append({
        "입력 텍스트": input_text,
        "추천 가게": recommended_stores,
    })

# 최종 결과 출력
for result2 in final_results:
    print(f"입력 텍스트: {result2['입력 텍스트']}")
    print("추천 가게:")
    for store in result2["추천 가게"]:
        print(f" - 가게명: {store['가게명']}, 유사도 점수: {store['유사도 점수']:.4f}")
    print()


result_df 컬럼들: Index(['입력 텍스트', '최종 선택 태그의 리스트'], dtype='object')
현재 처리 중인 입력 텍스트: 애완동물 동반 가능
현재 처리 중인 입력 텍스트: 해장하기 좋은 메뉴
현재 처리 중인 입력 텍스트: 룸이 있는 곳
현재 처리 중인 입력 텍스트: 단체로 가기 좋은 곳
현재 처리 중인 입력 텍스트: 기념일에 데이트하기 좋은 곳
현재 처리 중인 입력 텍스트: 조용하고 아늑한 분위기
현재 처리 중인 입력 텍스트: 든든한 백반
현재 처리 중인 입력 텍스트: 메뉴가 다양함
현재 처리 중인 입력 텍스트: 맵지 않은 음식
현재 처리 중인 입력 텍스트: 건강한 한 끼
입력 텍스트: 애완동물 동반 가능
추천 가게:
 - 가게명: 육쌈냉면 숙대점, 유사도 점수: 1.0000
 - 가게명: 미스터카츠 숙대입구점, 유사도 점수: 1.0000

입력 텍스트: 해장하기 좋은 메뉴
추천 가게:
 - 가게명: 샤브로21 숙명여대, 유사도 점수: 23.6000

입력 텍스트: 룸이 있는 곳
추천 가게:
 - 가게명: 더함, 유사도 점수: 25.6667
 - 가게명: 포라임 숙대본점, 유사도 점수: 17.6667
 - 가게명: 육쌈냉면 숙대점, 유사도 점수: 11.6667
 - 가게명: 베스트프렌드, 유사도 점수: 5.6667

입력 텍스트: 단체로 가기 좋은 곳
추천 가게:

입력 텍스트: 기념일에 데이트하기 좋은 곳
추천 가게:

입력 텍스트: 조용하고 아늑한 분위기
추천 가게:

입력 텍스트: 든든한 백반
추천 가게:

입력 텍스트: 메뉴가 다양함
추천 가게:
 - 가게명: 더함, 유사도 점수: 25.6667
 - 가게명: 포라임 숙대본점, 유사도 점수: 17.6667
 - 가게명: 육쌈냉면 숙대점, 유사도 점수: 11.6667
 - 가게명: 베스트프렌드, 유사도 점수: 5.6667

입력 텍스트: 맵지 않은 음식
추천 가게:

입력 텍스트: 건강한 한 끼
추천 가게:



# TF - IDF 리뷰 유사도 추출

In [63]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 입력 텍스트
input_txt = [
    '애완동물 동반 가능', '해장하기 좋은 메뉴', '룸이 있는 곳', '단체로 가기 좋은 곳',
    '기념일에 데이트하기 좋은 곳', '조용하고 아늑한 분위기', '든든한 백반',
    '메뉴가 다양함', '맵지 않은 음식', '건강한 한 끼'
]

df_t = pd.read_csv('/content/drive/MyDrive/result_meal.csv')

# NaN 값 제거 또는 대체
df_t["text_review"] = df_t["text_review"].fillna("")  # NaN 값을 빈 문자열로 대체

# TF-IDF 벡터라이저 초기화
tfidf_vectorizer = TfidfVectorizer()

# 데이터셋 리뷰 벡터화
corpus = df_t["text_review"].tolist()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# 결과 저장을 위한 리스트
result_data = []

# 각 입력 텍스트에 대해 유사도 계산
for user_input in input_txt:
    # 입력 텍스트 벡터화
    input_vector = tfidf_vectorizer.transform([user_input])

    # 코사인 유사도 계산
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix)

    # 유사도 추가
    df_t["Similarity"] = similarity_scores.flatten()

    # 장소별 유사도 상위 5개의 점수 평균 계산
    top5_avg_similarity = (
        df_t.groupby("store_name")
        .apply(lambda group: group.nlargest(10, "Similarity")["Similarity"].mean())
        .reset_index(name="Avg_Similarity")
    )

    # 평균 유사도를 기준으로 상위 5개의 장소
    top5_places = top5_avg_similarity.sort_values(by="Avg_Similarity", ascending=False).head(10)

    # 결과 저장
    result_data.append({"입력 텍스트": user_input, "상위 5개 장소 및 평균 유사도": top5_places.to_dict(orient="records")})

# 결과 데이터프레임 생성
result_df_2 = pd.DataFrame(result_data)

# 출력
print(result_df_2)

<ipython-input-63-ef4a3a9f805f>:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.nlargest(10, "Similarity")["Similarity"].mean())
<ipython-input-63-ef4a3a9f805f>:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.nlargest(10, "Similarity")["Similarity"].mean())
<ipython-input-63-ef4a3a9f805f>:41: DeprecationWarning: DataFrameGroupBy.apply operated on the

            입력 텍스트                                  상위 5개 장소 및 평균 유사도
0       애완동물 동반 가능  [{'store_name': '홍짜장 숙대점', 'Avg_Similarity': 0...
1       해장하기 좋은 메뉴  [{'store_name': '멘타미', 'Avg_Similarity': 0.049...
2          룸이 있는 곳  [{'store_name': '장재근초밥', 'Avg_Similarity': 0.0...
3      단체로 가기 좋은 곳  [{'store_name': '더함', 'Avg_Similarity': 0.0458...
4  기념일에 데이트하기 좋은 곳  [{'store_name': '멘타미', 'Avg_Similarity': 0.072...
5     조용하고 아늑한 분위기  [{'store_name': '미스터카츠 숙대입구점', 'Avg_Similarity...
6           든든한 백반  [{'store_name': '더함', 'Avg_Similarity': 0.0227...
7          메뉴가 다양함  [{'store_name': '하이퐁가든', 'Avg_Similarity': 0.0...
8         맵지 않은 음식  [{'store_name': '달볶이', 'Avg_Similarity': 0.034...
9          건강한 한 끼  [{'store_name': '몬스터플레이스', 'Avg_Similarity': 0...


<ipython-input-63-ef4a3a9f805f>:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.nlargest(10, "Similarity")["Similarity"].mean())
<ipython-input-63-ef4a3a9f805f>:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.nlargest(10, "Similarity")["Similarity"].mean())


In [64]:
# 유사도 점수가 높은 10개의 가게 선정 및 차등 가중치 적용
final_results = []

# 각 입력 텍스트에 대해 상위 10개 가게 선정 및 점수 계산
for result in result_data:
    user_input = result["입력 텍스트"]
    top_places = pd.DataFrame(result["상위 5개 장소 및 평균 유사도"])

    # 동일한 가게가 여러 입력 텍스트에 등장할 수 있으므로 모든 결과 합치기
    if not top_places.empty:
        # 상위 10개의 가게 및 평균 유사도 추출
        sorted_places = top_places.sort_values("Avg_Similarity", ascending=False).head(10)

        # 차등 가중치 계산: 10~5에서 0.5씩 감소
        weights = [200 - i * 10 for i in range(len(sorted_places))]

        # 최종 점수 계산: 유사도 × 가중치
        sorted_places["Weighted_Score"] = sorted_places["Avg_Similarity"] * weights

        # 최종 점수 기준으로 정렬
        sorted_places = sorted_places.sort_values("Weighted_Score", ascending=False)

        # 결과 저장
        final_results.append({
            "입력 텍스트": user_input,
            "추천 가게": sorted_places[["store_name", "Weighted_Score"]].to_dict(orient="records")
        })

# 최종 결과 출력
for result in final_results:
    print(f"입력 텍스트: {result['입력 텍스트']}")
    print("추천 가게:")
    for store in result["추천 가게"]:
        print(f" - 가게명: {store['store_name']}, 점수: {store['Weighted_Score']:.2f}")
    print()

입력 텍스트: 애완동물 동반 가능
추천 가게:
 - 가게명: 홍짜장 숙대점, 점수: 6.54
 - 가게명: 타코비 숙명여대점, 점수: 3.18
 - 가게명: 하이퐁가든, 점수: 2.50
 - 가게명: 샐러디 숙명여대점, 점수: 0.00
 - 가게명: 아리랑노점 숙대점, 점수: 0.00
 - 가게명: 육쌈냉면 숙대점, 점수: 0.00
 - 가게명: 원동미나리삼겹살, 점수: 0.00
 - 가게명: 온센 용산구점, 점수: 0.00
 - 가게명: 오복함흥냉면, 점수: 0.00
 - 가게명: 옛날감자전, 점수: 0.00

입력 텍스트: 해장하기 좋은 메뉴
추천 가게:
 - 가게명: 멘타미, 점수: 9.97
 - 가게명: 숙대소반, 점수: 7.31
 - 가게명: 마시앤바시, 점수: 6.01
 - 가게명: 스시 사이꼬, 점수: 5.57
 - 가게명: 나폴리키친, 점수: 3.76
 - 가게명: 백채김치찌개 숙대입구점, 점수: 3.21
 - 가게명: 윤지식당, 점수: 2.78
 - 가게명: 베나레스 숙대점, 점수: 2.55
 - 가게명: 카페시바, 점수: 2.32
 - 가게명: 두리식당, 점수: 2.08

입력 텍스트: 룸이 있는 곳
추천 가게:
 - 가게명: 장재근초밥, 점수: 5.64
 - 가게명: 하이퐁가든, 점수: 4.49
 - 가게명: 백채김치찌개 숙대입구점, 점수: 4.06
 - 가게명: 베나레스 숙대점, 점수: 3.34
 - 가게명: 만나칼국수, 점수: 2.81
 - 가게명: 긴자료코 숙대점, 점수: 2.61
 - 가게명: 빨봉분식 숙명여대점, 점수: 2.12
 - 가게명: 사이공마켓 숙대점, 점수: 1.97
 - 가게명: 리또리또 숙대점, 점수: 1.74
 - 가게명: 상록수, 점수: 1.50

입력 텍스트: 단체로 가기 좋은 곳
추천 가게:
 - 가게명: 더함, 점수: 9.18
 - 가게명: 베나레스 숙대점, 점수: 6.69
 - 가게명: 멘타미, 점수: 5.35
 - 가게명: 스시 사이꼬, 점수: 4.27
 - 가게명: 숙대소반, 점수: 3.37
 - 가게명

In [65]:
# 입력 텍스트별 최종 유사도 점수 계산
final_results_by_input = {}

# result와 result2의 점수 합산
for result, result2 in zip(final_results, final_results):
    input_text = result["입력 텍스트"]

    # 입력 텍스트에 대한 각 가게의 점수 합산
    store_scores = {}

    # result에서 점수 합산
    for store in result["추천 가게"]:
        store_name = store.get("store_name", store.get("가게명"))
        score = store.get("유사도 점수", store.get("Weighted_Score"))
        if store_name and score:
            if store_name not in store_scores:
                store_scores[store_name] = 0
            store_scores[store_name] += score

    # result2에서 점수 합산
    for store in result2["추천 가게"]:
        store_name = store.get("store_name", store.get("가게명"))
        score = store.get("유사도 점수", store.get("Weighted_Score"))
        if store_name and score:
            if store_name not in store_scores:
                store_scores[store_name] = 0
            store_scores[store_name] += score

    # 입력 텍스트별로 상위 5곳 추출
    sorted_stores = sorted(store_scores.items(), key=lambda x: x[1], reverse=True)[:5]
    final_results_by_input[input_text] = sorted_stores

# 결과 출력
for input_text, top_stores in final_results_by_input.items():
    print(f"입력 텍스트: {input_text}")
    print("최종 유사도 점수 상위 5곳:")
    for store_name, score in top_stores:
        print(f" - 가게명: {store_name}, 최종 점수: {score:.4f}")
    print()

입력 텍스트: 애완동물 동반 가능
최종 유사도 점수 상위 5곳:
 - 가게명: 홍짜장 숙대점, 최종 점수: 13.0845
 - 가게명: 타코비 숙명여대점, 최종 점수: 6.3505
 - 가게명: 하이퐁가든, 최종 점수: 5.0026

입력 텍스트: 해장하기 좋은 메뉴
최종 유사도 점수 상위 5곳:
 - 가게명: 멘타미, 최종 점수: 19.9441
 - 가게명: 숙대소반, 최종 점수: 14.6275
 - 가게명: 마시앤바시, 최종 점수: 12.0280
 - 가게명: 스시 사이꼬, 최종 점수: 11.1468
 - 가게명: 나폴리키친, 최종 점수: 7.5105

입력 텍스트: 룸이 있는 곳
최종 유사도 점수 상위 5곳:
 - 가게명: 장재근초밥, 최종 점수: 11.2850
 - 가게명: 하이퐁가든, 최종 점수: 8.9777
 - 가게명: 백채김치찌개 숙대입구점, 최종 점수: 8.1124
 - 가게명: 베나레스 숙대점, 최종 점수: 6.6843
 - 가게명: 만나칼국수, 최종 점수: 5.6284

입력 텍스트: 단체로 가기 좋은 곳
최종 유사도 점수 상위 5곳:
 - 가게명: 더함, 최종 점수: 18.3571
 - 가게명: 베나레스 숙대점, 최종 점수: 13.3881
 - 가게명: 멘타미, 최종 점수: 10.6908
 - 가게명: 스시 사이꼬, 최종 점수: 8.5369
 - 가게명: 숙대소반, 최종 점수: 6.7367

입력 텍스트: 기념일에 데이트하기 좋은 곳
최종 유사도 점수 상위 5곳:
 - 가게명: 멘타미, 최종 점수: 29.1477
 - 가게명: 스시 사이꼬, 최종 점수: 23.4123
 - 가게명: 숙대소반, 최종 점수: 18.5967
 - 가게명: 백채김치찌개 숙대입구점, 최종 점수: 17.4233
 - 가게명: 윤지식당, 최종 점수: 15.2306

입력 텍스트: 조용하고 아늑한 분위기
최종 유사도 점수 상위 5곳:
 - 가게명: 미스터카츠 숙대입구점, 최종 점수: 19.6521
 - 가게명: 쌍대포소금구이 본점, 최종 점수: 9.5899
 - 가게명: